In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import plot_tree

In [2]:
raw_data      = pd.read_csv('../data/sentence-relations/train.csv', index_col='id')
raw_submissions = pd.read_csv('../data/sentence-relations/test.csv', index_col='id')

# Only take english
raw_data = raw_data[raw_data['lang_abv'] == 'en']

training_data, test_data = train_test_split(raw_data, test_size=0.2, random_state=42)

training_data.head()

,premise,hypothesis,lang_abv,language,label
id,,,,,
b6e01c1a07,"Also, the Holy Family are said to have shelter...",The Holy family spent a total of three days here.,en,English,1
b8fa1a0044,Participants generally viewed the new internal...,Those organizations affected by the Sarbanes-O...,en,English,0
c828f51ef6,With a little practice almost anyone can flip ...,Practicing lets you do anything you put your m...,en,English,1
b2c98d5a99,More reserved and remote but a better administ...,The uncle had no match in administration; cert...,en,English,2
9cd35fee05,The company later told us that it had disconti...,The company later told us that it had enhanced...,en,English,2


In [3]:
from transformers import DataCollatorWithPadding
from transformers import AutoTokenizer

roberta_tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased", truncation=True)
data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

/home/alex/Documents/Studium/2-Master/ki-lab-ss23/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alex/Documents/Studium/2-Master/ki-lab-ss23/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
from torch.utils.data import Dataset, DataLoader

class BertSentenceDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

        self.X = self.data['premise'] + '[SEP]' + self.data['hypothesis']
        self.y = self.data['label']

        self.encoded = self.tokenizer(self.X.tolist(), padding=True, truncation=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encoded['input_ids'][idx],
            'attention_mask': self.encoded['attention_mask'][idx],
            'label': self.y.iloc[idx]
        }

bert_train_dataset = BertSentenceDataset(training_data, roberta_tokenizer)
bert_test_dataset = BertSentenceDataset(test_data, roberta_tokenizer)

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}
num_labels = len(id2label)

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id)

# Freeze the model except
modfication_layers = ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']

for name, param in model.named_parameters():
    if not any(layer in name for layer in modfication_layers):
        param.requires_grad = False

/home/alex/Documents/Studium/2-Master/ki-lab-ss23/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

training_args = TrainingArguments(
    output_dir="roberta-base-sentence-relation",
    learning_rate=1e-3,
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=30,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # fp16=True
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bert_train_dataset,
    eval_dataset=bert_test_dataset,
    tokenizer=roberta_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

  3%|▎         | 22/660 [00:22<09:30,  1.12it/s]







                                       
                                                
  3%|▎         | 22/660 [00:27<09:30,  1.12it/s]


{'eval_loss': 1.063098430633545, 'eval_accuracy': 0.4366812227074236, 'eval_f1': 0.4070744816125707, 'eval_precision': 0.4326339325367261, 'eval_recall': 0.4288465070460162, 'eval_runtime': 4.3319, 'eval_samples_per_second': 317.179, 'eval_steps_per_second': 1.385, 'epoch': 1.0}


  7%|▋         | 44/660 [00:50<09:12,  1.11it/s]







                                       
                                                
  7%|▋         | 44/660 [00:54<09:12,  1.11it/s]


{'eval_loss': 1.0937672853469849, 'eval_accuracy': 0.3937409024745269, 'eval_f1': 0.3253555493041426, 'eval_precision': 0.4178565600979394, 'eval_recall': 0.3884380774282598, 'eval_runtime': 4.3309, 'eval_samples_per_second': 317.256, 'eval_steps_per_second': 1.385, 'epoch': 2.0}


 10%|█         | 66/660 [01:18<08:52,  1.11it/s]







                                       
                                                
 10%|█         | 66/660 [01:22<08:52,  1.11it/s]


...

100%|██████████| 660/660 [13:45<00:00,  1.11it/s]







                                       
                                                 
100%|██████████| 660/660 [13:50<00:00,  1.11it/s]


{'eval_loss': 1.0762273073196411, 'eval_accuracy': 0.44905385735080056, 'eval_f1': 0.44523288906185643, 'eval_precision': 0.44846041987413204, 'eval_recall': 0.44673857387742383, 'eval_runtime': 4.3647, 'eval_samples_per_second': 314.796, 'eval_steps_per_second': 1.375, 'epoch': 30.0}



100%|██████████| 660/660 [13:50<00:00,  1.26s/it]

{'train_runtime': 830.6518, 'train_samples_per_second': 198.495, 'train_steps_per_second': 0.795, 'train_loss': 0.9703804247307055, 'epoch': 30.0}


TrainOutput(global_step=660, training_loss=0.9703804247307055, metrics={'train_runtime': 830.6518, 'train_samples_per_second': 198.495, 'train_steps_per_second': 0.795, 'total_flos': 1.011027844695312e+16, 'train_loss': 0.9703804247307055, 'epoch': 30.0})

In [ ]:
# Get the best model
model = AutoModelForSequenceClassification.from_pretrained('roberta-base-sentence-relation/checkpoint-374', num_labels=num_labels, id2label=id2label, label2id=label2id)

# Evaluate the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bert_train_dataset,
    eval_dataset=bert_test_dataset,
    tokenizer=None,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

100%|██████████| 43/43 [00:11<00:00,  3.61it/s]


{'eval_loss': 1.0233200788497925,
 'eval_accuracy': 0.5029112081513828,
 'eval_f1': 0.49280351950455037,
 'eval_precision': 0.5022752603866406,
 'eval_recall': 0.4993200260468844,
 'eval_runtime': 12.2102,
 'eval_samples_per_second': 112.529,
 'eval_steps_per_second': 3.522}